# Reduce GIWAXS Transformation

In [ ]:
# Cell 1

from pathlib import Path

import fabio
import numpy as np
import matplotlib.pylab as plt
import gixpy
from matplotlib.colors import LogNorm
plt.style.use('gixpy.style')

title = ""          # adds this to the title of the plots
path = Path.cwd()

IM_SIZE = (6.3, 3)  # inches

data_file = "data_file_name.edf"

flat_field_file = "flat-field_file_name.edf"

data = fabio.open(path / data_file)
header = data.header
print("\nEDF header:")
print(header)
raw_stitch = data.data.astype(np.float64)
flat_field = fabio.open(path / flat_field_file).data.astype(np.float64)

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(raw_stitch, norm=LogNorm(1, np.max(raw_stitch)))
ax.set_title("Stitched Data")
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7, label="counts")
fig.tight_layout()

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(flat_field)
ax.set_title("Flat Field")
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7, label="relative sensitivity")
fig.tight_layout()

## Adust pixel weights for presentation of stitch

In [ ]:
# Cell 2

adjusted_stitch = raw_stitch / flat_field
adjusted_stitch[np.where(adjusted_stitch == np.inf)] = 0
adjusted_stitch = np.nan_to_num(adjusted_stitch)

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(adjusted_stitch, norm=LogNorm(1, np.max(adjusted_stitch)))
ax.set_title(title)
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7)
fig.tight_layout()
fig.savefig("stitched_image.png", dpi=300, bbox_inches="tight")

del adjusted_stitch

## Find beam center and tilt

Get `det_dist` from specular scan to create default azimuthalIntegrator

In [ ]:
# Cell 3

%matplotlib widget

det_dist = header["DetectorDistance(m)"]
incident_angle = header["IncidentAngle(deg)"]
ai = gixpy.poni.new(det_dist, poni1=0.012625, poni2=.075, shape=raw_stitch.shape)
# print(ai._dist)
print(ai)
print("")

beam_finder = gixpy.poni.Nudger(ai, raw_stitch, flat_field=flat_field, incident_angle=incident_angle, radii=[12, 5])

Nudge beam center by number of pixels until the horizon line matches where the data is occluded.

Run this next cell to adjust plot above until the beam center is correct.

In [ ]:
# Cell 4

print(f"Brightest pixel has {beam_finder.data.max()} counts")
tilt_angle = 0.15   # degrees

# these numbers correspond to lattice plane spacings in angstrom
beam_finder.radii = [13.3, 10, 7, 5.2, 4.85, 4.5]   

beam_finder.set_tilt(tilt_angle)

beam_finder.set_nudge(
    0,   # nudge up
    0    # nudge right
)
beam_finder.show(12)

Save the PONI file once nudging is complete

In [ ]:
# Cell 5

%matplotlib inline

beam_finder.save(path / "cal.poni", orientation=2)

del beam_finder
plt.show()

## Transform

In [ ]:
# Cell 6

transformer = gixpy.GIXS(float(incident_angle), tilt_angle, Path("cal.poni"))
data_trans, flat_trans = transformer.transform(raw_stitch, flat_field, waveguiding=True)
transformed_filename = data_file.name.strip("_data.edf")
transformer.save_edf(transformed_filename)

print("Transformed geometry")
print(transformer.ai)
print("")

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(data_trans, norm=LogNorm(1, np.max(data_trans)))
ax.set_title("Stitched Data")
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7, label="counts")
fig.tight_layout()

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(flat_trans)
ax.set_title("Flat Field")
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7, label="relative sensitivity")
fig.tight_layout()

`ctrl + backtick` to open terminal

following cell will copy a command into clipboard, and then paste it to open mask maker.

In [ ]:
# Cell 7

import subprocess

path_to_image = str(path / (transformed_filename + "_data_transformed.edf"))
path_to_detector = str(path / "detector.h5")
to_copy = f'pyFAI-calib2 "{path_to_image}" -D "{path_to_detector}"'
try:
    subprocess.run("clip", text=True, input=to_copy)
except:
    print("Failed to copy to clipboard automatically. Do it manually:")
print(to_copy)

In [ ]:
# Cell 8

adjusted_trans = data_trans / flat_trans
adjusted_trans[np.where(adjusted_trans == np.infty)] = 0
adjusted_trans = np.nan_to_num(adjusted_trans)

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(adjusted_trans, norm=LogNorm(1, np.max(adjusted_trans)))
ax.set_title(title)
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7)
fig.tight_layout()
fig.savefig("transformed_image.png", dpi=300, bbox_inches="tight")

del adjusted_trans

## Make a mask

Run `pyFAI-calib2` to make a mask

In [ ]:
# Cell 9

transformer.load_mask(path / "mask.edf")

fig, ax = plt.subplots(1, 1, figsize=IM_SIZE, facecolor="w")
pos = ax.imshow(transformer.mask)
ax.set_title(title)
ax.set_xlabel("column (pixels)")
ax.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax, shrink=0.7)
fig.tight_layout()

## Cake

In [ ]:
# Cell 10

chi_range = (-180, 180)     # negative is the top side

cake = transformer.integrate2d(
    q_bins=500, 
    azimuthal_bins=360, 
    radial_range=None,
    azimuth_range=chi_range
)

print(cake[0].max())
print(np.sum(cake[0]))

fig, ax = plt.subplots(1, 1, figsize=(4, 3), facecolor="w")
pos = ax.imshow(cake[0], norm=LogNorm(1, np.max(cake[0])),
                 extent=(np.min(cake[1]), np.max(cake[1]), np.min(cake[2]), np.max(cake[2])),
                 aspect='auto')
fig_title = "Cake"
if title:
    fig_title = title + "\n" + fig_title
ax.set_title(fig_title)
ax.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax.set_ylabel(r"$\psi\ (\degree)$")
fig.colorbar(pos, ax=ax)
fig.tight_layout()
fig.savefig(f"{title.replace('@', 'a')}-cake.png".lstrip("-"), dpi=300, bbox_inches="tight")

# Sectors

In [ ]:
# Cell 11

def check_and_unlink(sector_name, range):
    sector_name += "_({},{}).edf".format(*range)
    if (transformer.dir / "sectors" / sector_name).is_file():
        (transformer.dir / "sectors" / sector_name).unlink()

## Full sector

In [ ]:
# Cell 12

azimuthal_range = (0, 180)
exposure_time = header["ExposureTime(s)"]

sector_name = "sector_full"
check_and_unlink(sector_name, azimuthal_range)

redu = transformer.sector(
    file_to_save=sector_name,
    q_range=None,
    azimuth_range=azimuthal_range,
    exposure_time=exposure_time,
    q_bins=1000
)

fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.scatter(
    redu[0], redu[1],
    s=5,  # marker size
    marker="o",  # marker shape
    edgecolors="black",  # marker edge color
    lw=.75,  # marker edge width
    alpha=1,  # transparency
    facecolor='w'  # marker face color
)


ax.set_title(title + ": ${}\\degree-{}\\degree$ reduction".format(*azimuthal_range))
ax.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax.grid()
ax.set_ylabel(r"Intensity (counts / (min $\cdot$ apparent pixel)")
ax.set_yscale("log")
fig.savefig(f"sectors/{title.replace('@', 'a')}-reduction-full.png".lstrip("-"), dpi=500, bbox_inches="tight")